# 4. Autograd

In [ ]:
import torch

## Setting `requires_grad`

In [ ]:
x = torch.ones(2, 2)
print(x)

In [ ]:
print(x.requires_grad)

In [ ]:
x.requires_grad_(True)  # note the underscore

In [ ]:
print(x.requires_grad)

### Checking how Autograd tracks operations

In [ ]:
y = x + 2
print(y)

In [ ]:
print(y.requires_grad)

In [ ]:
print(y.grad_fn)

In [ ]:
z = y.mean()
print(z)

---
## Back to our example :
$loss = (x \cdot W + b - y)^{2}$

In [ ]:
x = torch.Tensor([1,2,3])
y = torch.Tensor([1])

W = torch.rand((3,1), requires_grad=True)
b = torch.rand(1, requires_grad=True)
print(W, "\n\n", b)

In [ ]:
loss = (x @ W + b - y) ** 2
print(loss)

In [ ]:
print(W.grad, b.grad)

In [ ]:
loss.backward()

In [ ]:
print(W.grad, "\n\n", b.grad)

In [ ]:
print(x.grad, y.grad)

#### Gradients accumulate !

In [ ]:
loss = (x @ W + b - y) ** 2
loss.backward()

In [ ]:
print(W.grad, "\n\n", b.grad)

---
## Gradient for models parameters

In [ ]:
neural_net = torch.nn.Sequential(
    torch.nn.Linear(5, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 2),
)

loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
neural_net[0]

In [ ]:
neural_net[0].weight

A `Parameter` has its `requires_grad` set to `True` by default.

In [ ]:
x = torch.rand(15, 5) # batch_size, input_size
y = torch.randint(1, (15,)) # batch_size, output_size == num_classes

In [ ]:
predictions = neural_net(x)
loss = loss_fn(predictions, y)
print(loss)

In [ ]:
print(neural_net[0].weight.grad)

In [ ]:
loss.backward()

In [ ]:
print(neural_net[0].weight.grad)

In [ ]:
neural_net.zero_grad()

In [ ]:
print(neural_net[0].weight.grad)

---
## Stop the history tracking

Useful during inference

In [ ]:
x = torch.ones(2, 2, requires_grad=True)

In [ ]:
y = x ** 2
print(y.requires_grad)

In [ ]:
with torch.no_grad():
    z = x ** 2
    print(z.requires_grad)

In [ ]:
print(z.grad, z.grad_fn)

---
# Building our training loop (4 / 5)

In [ ]:
# INITIALIZATION

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, RandomCrop
from torchvision.datasets import ImageFolder

device = torch.device("cpu")

transform = Compose((RandomCrop((50, 50)), ToTensor()))
dataset = ImageFolder(root="../alien-vs-predator/", transform=transform)
loader = DataLoader(dataset, batch_size=5, shuffle=True)

model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(7500, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 2),
)
model.to(device)

loss_fn = nn.CrossEntropyLoss()

In [ ]:
# TRAINING LOOP

for samples, labels in loader:
    samples = samples.to(device)
    labels = labels.to(device)
    predictions = model(samples)
    loss = loss_fn(predictions, labels)
    loss.backward()
    # update model parameters
    model.zero_grad() #